In [3]:
from pymongo import MongoClient
import json

with open('data/credentials_mongodb.json') as f:
    login = json.load(f)

client = MongoClient(**login)

In [4]:
my_db = client['my_db']
my_db

Database(MongoClient(host=['ac-tiwkcrk-shard-00-01.ioin6ns.mongodb.net:27017', 'ac-tiwkcrk-shard-00-00.ioin6ns.mongodb.net:27017', 'ac-tiwkcrk-shard-00-02.ioin6ns.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-12y3av-shard-0', tls=True), 'my_db')

In [5]:
client.list_database_names()

['sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

In [6]:
client['sample_mflix'].list_collection_names()

['theaters', 'sessions', 'comments', 'movies', 'users']

### Retrieve a random document associated with some movie/TV series produced in 2000.

In [27]:
list(
    client['sample_mflix']['movies'].find( {'year': 2000} , limit=1)
)

[{'_id': ObjectId('573a139af29313caabcf0782'),
  'fullplot': 'Set in Hong Kong, 1962, Chow Mo-Wan is a newspaper editor who moves into a new building with his wife. At the same time, Su Li-zhen, a beautiful secretary and her executive husband also move in to the crowded building. With their spouses often away, Chow and Li-zhen spend most of their time together as friends. They have everything in common from noodle shops to martial arts. Soon, they are shocked to discover that their spouses are having an affair. Hurt and angry, they find comfort in their growing friendship even as they resolve not to be like their unfaithful mates.',
  'imdb': {'rating': 8.1, 'votes': 67663, 'id': 118694},
  'year': 2000,
  'plot': 'Two neighbors, a woman and a man, form a strong bond after both suspect extramarital activities of their spouses. However, they agree to keep their bond platonic so as not to commit similar wrongs.',
  'genres': ['Drama', 'Romance'],
  'rated': 'PG',
  'metacritic': 85,
  't

### Retrieve only the title and cast of movies produced in 2005, limit results to 3 documents.


In [28]:
list(
    client['sample_mflix']['movies'].find( {'year': 2010} ,  projection = {'_id':0, 'title':1, 'cast':1}, limit=3) 
)

[{'cast': ['èva Gèbor', 'Istvèn Znamenèk', 'èkos Horvèth', 'Lia Pokorny'],
  'title': 'Pèl Adrienn'},
 {'title': 'In My Sleep',
  'cast': ['Philip Winchester',
   'Tim Draxl',
   'Lacey Chabert',
   'Abigail Spencer']},
 {'cast': ['James Badge Dale',
   'Joseph Mazzello',
   'Jon Seda',
   'Sebastian Bertoli'],
  'title': 'The Pacific'}]

### Retrieve the number of movies and TV series with a Metacritic rating of exactly 90 for each year from 2000 to 2005 (inclusive).

In [29]:
for i in range(2000,2006):
    print(f"{i}: {client['sample_mflix']['movies'].count_documents(filter={'metacritic': 90,   'year' : i })}")

2000: 2
2001: 3
2002: 2
2003: 3
2004: 2
2005: 0


### Retrieve the title and runtime of the 10 shortest movies in the movies collection.



In [32]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'runtime': {
                '$exists': True 
            }
        },
        projection={'_id': 0, 'title': 1, 'runtime': 1},
        sort=[('runtime', 1)],
        limit=10,
    )
)

[{'runtime': 1, 'title': 'Neko no shukai'},
 {'runtime': 1, 'title': 'The Kiss'},
 {'runtime': 1, 'title': 'Newark Athlete'},
 {'runtime': 1, 'title': 'Dickson Experimental Sound Film'},
 {'runtime': 1, 'title': 'Blacksmith Scene'},
 {'runtime': 1, 'title': 'The Kiss'},
 {'runtime': 2, 'title': 'Luxo Jr.'},
 {'runtime': 2, 'title': 'Andrè and Wally B.'},
 {'runtime': 2, 'title': 'Game Over'},
 {'runtime': 2, 'title': 'Fresh Guacamole'}]

### Retrieve the title, production year, and number of awards of all movies that

- have been produced between 1950 and 2000 (inclusive)
- have an IMDB rating of 8.5 or better
- won at least 30 awards.

and sort the results by production year in descending order.

In [12]:
list(
client["sample_mflix"]["movies"].find(
        filter={
            'year': {
                '$gte': 1950,
                '$lte': 2000
            },  'imdb.rating' : { '$gte' :8.5 },  'awards.wins' : { '$gte' : 30 } 
        } , projection={'_id': 0, 'title': 1, 'year': 1, 'awards.wins': 1},
        sort=[('year', -1)]
)
)

[{'year': 2000, 'title': 'Gladiator', 'awards': {'wins': 63}},
 {'year': 2000, 'title': 'Memento', 'awards': {'wins': 54}},
 {'year': 1999, 'title': 'The Matrix', 'awards': {'wins': 37}},
 {'year': 1998, 'title': 'Saving Private Ryan', 'awards': {'wins': 83}},
 {'year': 1997, 'title': 'Life Is Beautiful', 'awards': {'wins': 66}},
 {'title': 'Life Is Beautiful', 'awards': {'wins': 66}, 'year': 1997},
 {'year': 1995, 'title': 'The Usual Suspects', 'awards': {'wins': 36}},
 {'year': 1995, 'title': 'Se7en', 'awards': {'wins': 32}},
 {'year': 1994, 'title': 'Pulp Fiction', 'awards': {'wins': 64}},
 {'year': 1994, 'title': 'Forrest Gump', 'awards': {'wins': 46}},
 {'title': "Schindler's List", 'awards': {'wins': 81}, 'year': 1993},
 {'year': 1991, 'title': 'The Silence of the Lambs', 'awards': {'wins': 56}},
 {'year': 1990, 'title': 'Goodfellas', 'awards': {'wins': 43}},
 {'year': 1981, 'title': 'Raiders of the Lost Ark', 'awards': {'wins': 32}},
 {'year': 1977,
  'title': 'Star Wars: Episod

### Find the top 15 highest-rated movies according to IMDB for movies that have at least 100,000 votes. Return only the title, year, and imdb.rating fields.

In [13]:
list(
client["sample_mflix"]["movies"].find(
        filter={
            'imdb.votes': {'$gte': 100000}
        } , projection={'_id': 0, 'title': 1, 'year': 1, 'imdb.rating': 1},
        sort=[('imdb.rating', -1)],
        limit=15
)
)

[{'title': 'Band of Brothers', 'year': 2001, 'imdb': {'rating': 9.6}},
 {'imdb': {'rating': 9.3}, 'year': 1994, 'title': 'The Shawshank Redemption'},
 {'imdb': {'rating': 9.3}, 'year': 1994, 'title': 'The Shawshank Redemption'},
 {'imdb': {'rating': 9.2}, 'year': 1972, 'title': 'The Godfather'},
 {'imdb': {'rating': 9.1}, 'year': 1974, 'title': 'The Godfather: Part II'},
 {'imdb': {'rating': 9.0}, 'year': 2008, 'title': 'The Dark Knight'},
 {'imdb': {'rating': 8.9},
  'year': 1966,
  'title': 'The Good, the Bad and the Ugly'},
 {'title': "Schindler's List", 'year': 1993, 'imdb': {'rating': 8.9}},
 {'title': '12 Angry Men', 'year': 1957, 'imdb': {'rating': 8.9}},
 {'imdb': {'rating': 8.9}, 'year': 1994, 'title': 'Pulp Fiction'},
 {'imdb': {'rating': 8.9}, 'year': 1999, 'title': 'Fight Club'},
 {'imdb': {'rating': 8.9},
  'year': 2003,
  'title': 'The Lord of the Rings: The Return of the King'},
 {'imdb': {'rating': 8.8}, 'year': 1994, 'title': 'Forrest Gump'},
 {'imdb': {'rating': 8.8},

### Retrieve the title, production year, and IMDB rating of movies in which both **Morgan Freeman** and **Clint Eastwood** played a role (among other actors in those movies). Sort the returned documents by year in descending order.

In [14]:
list(
client["sample_mflix"]["movies"].find(
        filter={
            'cast': {'$all': ['Morgan Freeman','Clint Eastwood']}
        } , projection={'_id': 0, 'title': 1, 'year': 1, 'imdb.rating': 1},
        sort=[('imdb.rating', -1)]
)
)

[{'imdb': {'rating': 8.3}, 'year': 1992, 'title': 'Unforgiven'},
 {'imdb': {'rating': 8.1}, 'year': 2004, 'title': 'Million Dollar Baby'}]

### Retrieve documents associated with movies which:

- are available in both German and French (among other languages), but not in English,
- are either rated above 8 according to IMDB, or above 7.5 according to the critic ratings of Rotten Tomatoes,
- have at least 50 Rotten Tomatoes critic reviews.

return only the title, year, IMDB rating, Rotten Tomatoes critic rating and country of production fields. Sort the results by IMDB rating in descending order.


In [15]:
list(
client["sample_mflix"]["movies"].find(
        filter={
            '$and' :[ {'languages': {'$all': ['German','French'],  '$exists': True}}, {'languages': {'$nin': ['English'],  '$exists': True} }] ,  
            '$or' :[ {'imdb.rating': {'$gte': 8}},   {'tomatoes.critic.rating': {'$gte': 7.5}}  ],
            'tomatoes.critic.numReviews': {'$gte': 50}
        } , projection={'_id': 0, 'title': 1, 'year': 1, 'imdb.rating': 1, 'tomatoes.critic.rating':1, 'countries':1 },
        sort=[('imdb.rating', -1)]
)
)

[{'imdb': {'rating': 8.0},
  'year': 1958,
  'title': 'Elevator to the Gallows',
  'tomatoes': {'critic': {'rating': 8.1}},
  'countries': ['France']},
 {'imdb': {'rating': 7.8},
  'year': 2013,
  'title': 'The Wind Rises',
  'tomatoes': {'critic': {'rating': 7.9}},
  'countries': ['Japan']},
 {'title': 'The Wind Rises',
  'year': 2013,
  'imdb': {'rating': 7.8},
  'countries': ['Japan'],
  'tomatoes': {'critic': {'rating': 7.9}}},
 {'title': 'Sèraphine',
  'year': 2008,
  'imdb': {'rating': 7.4},
  'countries': ['France', 'Belgium'],
  'tomatoes': {'critic': {'rating': 7.5}}},
 {'title': '35 Shots of Rum',
  'year': 2008,
  'imdb': {'rating': 7.1},
  'countries': ['France', 'Germany'],
  'tomatoes': {'critic': {'rating': 8.1}}}]

### Find the title and production year of the top 20 award-winning movies which have **not** been produced in USA, Canada, UK, or Australia.


In [16]:
list(
client["sample_mflix"]["movies"].find(
        filter={
             'countries': {'$nin': ['USA','Canada','UK','Australia'], '$exists': True} 
        } , projection={'_id': 0, 'title': 1, 'year': 1},
        sort=[('awards.wins', -1)],
        limit=20
)
)

[{'title': 'The Artist', 'year': 2011},
 {'title': 'Blue Is the Warmest Color', 'year': 2013},
 {'title': 'Amour', 'year': 2012},
 {'title': 'Amour', 'year': 2012},
 {'title': 'A Separation', 'year': 2011},
 {'year': 2006, 'title': 'The Lives of Others'},
 {'title': 'Let the Right One In', 'year': 2008},
 {'year': 2002, 'title': 'City of God'},
 {'year': 2006, 'title': 'Volver'},
 {'title': 'The Sea Inside', 'year': 2004},
 {'year': 1997, 'title': 'Life Is Beautiful'},
 {'title': 'Life Is Beautiful', 'year': 1997},
 {'year': 2001, 'title': 'Amèlie'},
 {'year': 2009, 'title': 'The White Ribbon'},
 {'title': 'The Great Beauty', 'year': 2013},
 {'year': 2000, 'title': 'Amores Perros'},
 {'title': 'Shall We Dance?', 'year': 1996},
 {'year': 1999, 'title': 'All About My Mother'},
 {'year': 2013, 'title': 'The Grandmaster'},
 {'title': 'Shall We Dance?', 'year': 1996}]


### Find the title and production year of the top 20 award-winning movies which have **not** been produced in USA, Canada, UK, or Australia.


In [17]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {"$match": {"countries": {"$nin": ['USA', 'Canada', 'UK','Australia']}}},
            {'$sort':{'awards.wins':-1}},
            {'$project': {'_id': 0, 'title': 1,'year':1, 'imdbRating':'$imdb.rating'}},
           
            {'$limit':20}
        ]
    )
)

[{'title': 'The Artist', 'year': 2011, 'imdbRating': 8.0},
 {'title': 'Blue Is the Warmest Color', 'year': 2013, 'imdbRating': 7.9},
 {'title': 'Amour', 'year': 2012, 'imdbRating': 7.9},
 {'title': 'Amour', 'year': 2012, 'imdbRating': 7.9},
 {'title': 'A Separation', 'year': 2011, 'imdbRating': 8.4},
 {'year': 2006, 'title': 'The Lives of Others', 'imdbRating': 8.5},
 {'title': 'Let the Right One In', 'year': 2008, 'imdbRating': 8.0},
 {'year': 2002, 'title': 'City of God', 'imdbRating': 8.7},
 {'year': 2006, 'title': 'Volver', 'imdbRating': 7.6},
 {'title': 'The Sea Inside', 'year': 2004, 'imdbRating': 8.1},
 {'year': 1997, 'title': 'Life Is Beautiful', 'imdbRating': 8.6},
 {'title': 'Life Is Beautiful', 'year': 1997, 'imdbRating': 8.6},
 {'year': 2001, 'title': 'Amèlie', 'imdbRating': 8.4},
 {'year': 2009, 'title': 'The White Ribbon', 'imdbRating': 7.8},
 {'title': 'The Great Beauty', 'year': 2013, 'imdbRating': 7.7},
 {'year': 2000, 'title': 'Amores Perros', 'imdbRating': 8.1},
 {'t

### Count the number of movies (not TV series) produced in the USA per year, in and after the year 2000. Sort results by year in ascending order.

In [18]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {"$match": {"year": {"$gte": 2000}}},
            {"$match": {"countries": 'USA'}},
            {"$match": {"type": 'movie'}},
            {'$group': {
                '_id': '$year',
                'num_of_movies': {'$sum': 1}
            }}
            ,
            {'$sort':{'_id':1}}
        ]
    )
)

[{'_id': 2000, 'num_of_movies': 341},
 {'_id': 2001, 'num_of_movies': 318},
 {'_id': 2002, 'num_of_movies': 337},
 {'_id': 2003, 'num_of_movies': 298},
 {'_id': 2004, 'num_of_movies': 325},
 {'_id': 2005, 'num_of_movies': 357},
 {'_id': 2006, 'num_of_movies': 373},
 {'_id': 2007, 'num_of_movies': 423},
 {'_id': 2008, 'num_of_movies': 445},
 {'_id': 2009, 'num_of_movies': 416},
 {'_id': 2010, 'num_of_movies': 409},
 {'_id': 2011, 'num_of_movies': 458},
 {'_id': 2012, 'num_of_movies': 485},
 {'_id': 2013, 'num_of_movies': 546},
 {'_id': 2014, 'num_of_movies': 536},
 {'_id': 2015, 'num_of_movies': 230}]

In [19]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {"$match": {"year": {"$gte": 2000}}},
            {"$match": {"countries": 'USA'}},
            {"$match": {"type": 'movie'}},
            {'$group': {
                '_id': '$year',
                'num_of_movies': {'$sum': 1}
            }}
            ,
            {'$sort':{'_id':1}},
            {'$project': {'_id': 0, 'num_of_movies':1, 'prod_year':'$_id'}}
        ]
    )
)

[{'num_of_movies': 341, 'prod_year': 2000},
 {'num_of_movies': 318, 'prod_year': 2001},
 {'num_of_movies': 337, 'prod_year': 2002},
 {'num_of_movies': 298, 'prod_year': 2003},
 {'num_of_movies': 325, 'prod_year': 2004},
 {'num_of_movies': 357, 'prod_year': 2005},
 {'num_of_movies': 373, 'prod_year': 2006},
 {'num_of_movies': 423, 'prod_year': 2007},
 {'num_of_movies': 445, 'prod_year': 2008},
 {'num_of_movies': 416, 'prod_year': 2009},
 {'num_of_movies': 409, 'prod_year': 2010},
 {'num_of_movies': 458, 'prod_year': 2011},
 {'num_of_movies': 485, 'prod_year': 2012},
 {'num_of_movies': 546, 'prod_year': 2013},
 {'num_of_movies': 536, 'prod_year': 2014},
 {'num_of_movies': 230, 'prod_year': 2015}]

### Create an aggregation pipeline that lists the top 10 most common movie languages, along with the count of movies for each language. Sort results based on the count in descending order.

In [20]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {'$unwind': '$languages'},

            {'$group': {
                '_id': '$languages',
                'num_of_movies': {'$sum': 1}
            }}
            ,
            {'$sort':{'num_of_movies':-1}},
            {'$limit':10}
        ]
    )
)

[{'_id': 'English', 'num_of_movies': 16048},
 {'_id': 'French', 'num_of_movies': 2800},
 {'_id': 'Spanish', 'num_of_movies': 1787},
 {'_id': 'German', 'num_of_movies': 1583},
 {'_id': 'Italian', 'num_of_movies': 1362},
 {'_id': 'Japanese', 'num_of_movies': 1014},
 {'_id': 'Russian', 'num_of_movies': 919},
 {'_id': 'Mandarin', 'num_of_movies': 539},
 {'_id': 'Hindi', 'num_of_movies': 524},
 {'_id': 'Portuguese', 'num_of_movies': 388}]

### Create an aggregation pipeline that lists the top 15 prolific directors along with the number of movies they have directed. 

In [21]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {'$unwind': '$directors'},
            

            {'$group': {
                '_id': '$directors',
                'num_of_movies': {'$sum': 1},
                'avg_imdb_rating': {'$avg': '$imdb.rating'}
            }},
            { '$project': {'_id':1,'num_of_movies':1,'avg_imdb_rating': { '$round': [ "$avg_imdb_rating", 1 ]}}}
            ,
            {'$sort':{'num_of_movies':-1}},
            {'$limit':15}
        ]
    )
)

[{'_id': 'Woody Allen', 'num_of_movies': 40, 'avg_imdb_rating': 7.2},
 {'_id': 'John Ford', 'num_of_movies': 35, 'avg_imdb_rating': 7.2},
 {'_id': 'Takashi Miike', 'num_of_movies': 34, 'avg_imdb_rating': 6.9},
 {'_id': 'John Huston', 'num_of_movies': 34, 'avg_imdb_rating': 7.2},
 {'_id': 'Werner Herzog', 'num_of_movies': 33, 'avg_imdb_rating': 7.5},
 {'_id': 'Martin Scorsese', 'num_of_movies': 32, 'avg_imdb_rating': 7.6},
 {'_id': 'Alfred Hitchcock', 'num_of_movies': 31, 'avg_imdb_rating': 7.6},
 {'_id': 'Sidney Lumet', 'num_of_movies': 30, 'avg_imdb_rating': 7.0},
 {'_id': 'Michael Apted', 'num_of_movies': 29, 'avg_imdb_rating': 7.1},
 {'_id': 'George Cukor', 'num_of_movies': 29, 'avg_imdb_rating': 7.2},
 {'_id': 'Steven Spielberg', 'num_of_movies': 29, 'avg_imdb_rating': 7.5},
 {'_id': 'Mario Monicelli', 'num_of_movies': 29, 'avg_imdb_rating': 7.0},
 {'_id': 'Steven Soderbergh', 'num_of_movies': 28, 'avg_imdb_rating': 6.7},
 {'_id': 'Spike Lee', 'num_of_movies': 28, 'avg_imdb_rating'